In [1]:
from collections import defaultdict
from copy import deepcopy
from math import gcd

In [2]:
data = open("input/20").read().splitlines()

In [3]:
# low = False
# high = True

## Note

Assumption that the node pointing to rx is an conjunction, so reading out that and finding all nodes that point to that one

In [4]:
raw_nodes = {}
raw_mapping = defaultdict(list)
conjunctions = []
part2_conjunction = None
for line in data:   
    sender, reciever = line.split(" -> ")
    
    node_type = "flipflop" if "%" in sender else "conjunction"
    if sender != "broadcaster":
        if node_type == "flipflop":
            raw_nodes[sender[1:]] = [node_type, False, None]
        else:
            conjunctions.append(sender[1:])
            raw_nodes[sender[1:]] = [node_type, True, defaultdict(str)]
    
    for entry in reciever.split(","):
        raw_mapping[sender[1:]].append(entry.strip())
        
        if entry.strip() == "rx":
            part2_conjunction = sender[1:]

# Map up the conjunctions

In [5]:
part2_targets = []
for node, values in raw_mapping.items():
    for conjunction in values:
        if conjunction in conjunctions:
            if conjunction == part2_conjunction:
                part2_targets.append(node)
            raw_nodes[conjunction][2][node] = False

In [6]:
nodes = deepcopy(raw_nodes)
mapping = deepcopy(raw_mapping)

part2_map = {key: None for key in part2_targets}

def button(count):
    to_handle = []
    # removing first char in key so that's why it's missing b in broadcaster
    for key in mapping["roadcaster"]:
        to_handle.append([
            "broadcaster", False, key
        ])
    targets = []
    high_pulses = 0
    low_pulses = 1
    while True:
        if len(to_handle) == 0:
            return False, [high_pulses, low_pulses]
        new_to_handle = []
        for source, pulse, destination in to_handle:
            # To debug
            #print(f"\t{source} -{pulse}-> {destination}")
            
            if destination == part2_conjunction and pulse:
                if not part2_map[source]:
                    part2_map[source] = count
                if None not in part2_map.values():
                    return True, part2_map

            if pulse == True:
                high_pulses += 1
            else:
                low_pulses += 1
            
            # end node
            if destination not in nodes:
                continue
            
            type_, state, memory = nodes[destination]
            
            if type_ == "flipflop":
                if pulse == False:
                    if state == False:
                        nodes[destination][1] = True
                        pulse = True
                    else:
                        nodes[destination][1] = False
                        pulse = False
                else:
                    pulse = None
                    
            if type_ == "conjunction":
                nodes[destination][2][source] = pulse            
                all_vals = nodes[destination][2].values()         
                pulse = False in nodes[destination][2].values()
                      
            targets = mapping[destination]
            for target in targets:
                if pulse is not None:
                    new_to_handle.append([destination, pulse, target])
        to_handle = deepcopy(new_to_handle)

## Part 2
For part two get out all values that point to the conjunction before rx

Then find where they get "high" (True in my implementation).

Store and do LCM over them (the LCM in this case is unnessary as all my values are prime, but generalizes more)

In [7]:
def get_lcm(nums):
    lcm = 1
    for i in nums:
        lcm = lcm*i//gcd(lcm, i)
    return lcm

In [8]:
c = 1
num_high, num_low = 0, 0

while True:
    part2, res = button(c)
    
    # Starting at 1 so off by 1
    if not part2 and c < 1001:
        num_high += res[0]
        num_low += res[1]
    
    if part2:
        lcm = get_lcm(list(res.values()))
        break
        
    c += 1
    
print("Answer #1:", num_high * num_low)
print("Answer #2:", lcm)

Answer #1: 832957356
Answer #2: 240162699605221
